## Web Search Test
The **Web Search Test** ant demonstrates the use of a Web API secured using an API key with an auth header. This sample requires you to create a Web Search (bing) resource in Azure and enter its corresponding API key in [0-AI-settings](0-AI-settings.ipynb)

## First steps
- First, download AntRunLib from Nuget
- At least once, setup the enviroment using **[0-AI-settings](0-AI-settings.ipynb)**

In [1]:
#r "nuget: AntRunnerLib, 0.6.5"

using AntRunnerLib;
using AntRunnerLib.Identity;
using static AntRunnerLib.ClientUtility;
using System.IO;

#!import config/Settings.cs 

var envVariables = Settings.GetEnvironmentVariables();
foreach (var kvp in envVariables)
{
    Environment.SetEnvironmentVariable(kvp.Key, kvp.Value);
}

var config = AzureOpenAIConfigFactory.Get();
var client = GetOpenAIClient(config);


Installed Packages AntRunnerLib, 0.6.5

## Ensure the Web Search Test assistant exists
The definition of this assistant is located in the ".\AssistantDefinitions\Web Search Test" folder

".\AssistantDefinitions\" is a default path. You can override it by setting the **ASSISTANTS_BASE_FOLDER_PATH** enviroment variable.

"Web Search Test" contains the following files:
```
│   instructions.md - The instructions
│   manifest.json - The base assistant definition
│
└───OpenAPI
        api.bing.microsoft.com.json - A swagger file defining a tool named 'crawl' that uses web search
        auth.json - See below... 
```        

## auth.json
AntRunnerLib uses the instructions in this file to create the correct auth header(s) for the API. In this case a header is added to the web request, **Ocp-Apim-Subscription-Key**, the header's value is supplied by an environment variable named, SEARCH_API_KEY. The environment variable name is user defined and ensuring it has a value is the responsibility of the application. 

```
{
  "hosts": {
    "api.bing.microsoft.com": {
      "auth_type": "service_http",
      "header_name": "Ocp-Apim-Subscription-Key",
      "header_value_env_var": "SEARCH_API_KEY"
    }
  }
}
```

### Explanation
AssistantUtility.Create acts based on the contents of a folder structure following a convention laid out in these notebooks. In this case, it copied the contents of instructions.md into the assistant manifest.json. It also parsed the api.bing.microsoft.com swagger which follows an Open API schema into tool defintions for the assitant.

In [2]:
var assistantId = await AssistantUtility.Create("Web Search Test", config);
Console.WriteLine(assistantId)

asst_4EUSOW7dhpete4bUfLf1tVTG


## Run the Assistant

output.Dialog shows the conversation. 

In [3]:
var assistantRunOptions = new AssistantRunOptions() {
    AssistantName = "Web Search Test",
    Instructions = "Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary",
    UseConversationEvaluator = false
};
var output = await AntRunnerLib.AssistantRunner.RunThread(assistantRunOptions, config);
Console.WriteLine(output.Dialog)


User: Use python to get todays date and then use it to find things to do in Atlanta this weekend. Be careful to not include events in the past in your final answer. You can perform multiple searches if necessary

Assistant: I ran this code:
```
from datetime import datetime, timedelta

# Get today's date
today = datetime.now()
# Calculate the upcoming weekend dates
friday = today + timedelta((4-today.weekday()) % 7)  # Next Friday
saturday = friday + timedelta(1)  # Next Saturday
sunday = saturday + timedelta(1)  # Next Sunday

today_date = today.strftime('%Y-%m-%d')
friday_date = friday.strftime('%Y-%m-%d')
saturday_date = saturday.strftime('%Y-%m-%d')
sunday_date = sunday.strftime('%Y-%m-%d')

today_date, friday_date, saturday_date, sunday_date
```

I called the tool named crawl with these arguments:
```
{"q": "things to do in Atlanta on September 13, 2024", "textFormat": "HTML"}
```
and got this result:
```
{"_type": "SearchResponse", "queryContext": {"originalQuery": "things to do

In [4]:
Console.WriteLine(output.LastMessage)

Here's a list of things to do in Atlanta on the weekend of September 13-15, 2024:

### September 13, 2024 (Friday)
1. **Children’s Museum of Atlanta** - Every Friday until 09/30/2024.
2. **King Buzzo & Trevor Dunn Concert** - Hell at The Masquerade, 75 MLK Jr Dr SW, Atlanta, GA.
3. **Atlanta Food & Wine Festival** - Various events across the city.

### September 14, 2024 (Saturday)
1. **Pumpkin Festival at Stone Mountain Park** - Runs from 09/14/2024 to 11/03/2024.
2. **DIY Mocktail Workshop** - Featuring NoLiq Beverages.
3. **Farzad Farzin Live in Concert** - Cultural event.
4. **Sandy Springs Fall Arts & Crafts Festival** - Runs from 09/14/2024 to 09/15/2024.

### September 15, 2024 (Sunday)
1. **Atlanta Streets Alive** - West End to Grant Park, streets temporarily closed to traffic.
2. **Farm Fair** - Family-friendly event.
3. **Dreammile 2024** - Charity walk/run event.
4. **Five Finger Death Punch Concert** - Rock concert.
5. **Sandy Springs Fall Arts & Crafts Festival** - Continu